In [8]:
import os
import random
from pathlib import Path
import pandas as pd
import cv2
import yaml


In [9]:
def ConvertLisaToYolo(dataSetFolderPath: str,outputPath: str  ):
    # annotations = []
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=";")
    uniqueSigns = annotationsDataFrame['Annotation tag'].unique().tolist()
    uniqueFiles = annotationsDataFrame['Filename'].unique()
    for file in uniqueFiles:
        image_file = os.path.join(dataSetFolderPath, file)
        if not os.path.exists(image_file):
            continue
        fileDataFrame = annotationsDataFrame[annotationsDataFrame['Filename'] == file]
        fileAnnotations = []
        for index, row in fileDataFrame.iterrows():
            x1, y1, x2, y2 = row[2], row[3], row[4], row[5]
            image = cv2.imread(dataSetFolderPath+"/"+row[0])
            h, w, _ = image.shape
            xCenter = ((x1 + x2) / 2) / w
            yCenter = ((y1 + y2) / 2) / h
            width = (x2 - x1) / w
            height = (y2 - y1) / h
            classId = uniqueSigns.index(row[1])
            fileAnnotations.append(f"{classId} {xCenter} {yCenter} {width} {height}")
        with open(os.path.join(outputPath, f"{Path(file).stem}.txt"), "w") as f:
            f.write("\n".join(fileAnnotations))


In [ ]:
def CreateTrainValFiles(dataSetFolderPath: str):
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=";")
    uniqueSigns = annotationsDataFrame['Annotation tag'].unique().tolist()
    print(uniqueSigns)
    print(len(uniqueSigns))
    uniqueFiles = annotationsDataFrame['Filename'].unique().tolist()
    image_files = ["./DataSet/" + file for file in uniqueFiles ]
    random.shuffle(image_files)

    split_ratio = 0.8
    train_files = image_files[:int(len(image_files) * split_ratio)]
    val_files = image_files[int(len(image_files) * split_ratio):]

    with open("train.txt", "w") as f:
        f.writelines([ file + "\n" for file in train_files])

    with open("val.txt", "w") as f:
        f.writelines([file + "\n" for file in val_files])
    return len(uniqueSigns), uniqueSigns
    

In [ ]:
def CreateYaml(fileName: str, numClass: int, classList: list):
    data = {
    'train': './train.txt',
    'val': './val.txt',
    'nc': numClass,
    'names': classList
    }

    with open(fileName, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

In [ ]:
os.mkdir('./Yolo Data Annotation')

In [12]:
ConvertLisaToYolo('./DataSet', './Yolo Data Annotation')
numClass, classList = CreateTrainValFiles('./DataSet')
CreateYaml('signboard.yaml', numClass, classList)